In [ ]:
# !python -m venv venv
# # !venv/scripts/activate
!pip install pandas numpy matplotlib seaborn plotly geopandas scipy scikit-learn statsmodels requests streamlit streamlit_jupyter SQLAlchemy psycopg2-binary pyspark

In [ ]:
# Data Manipulation
import pandas as pd
import numpy as np

# Data Visualization
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import geopandas as gpd

# Scientific Computing
import scipy

# Machine Learning
import sklearn

# Statistical Models
import statsmodels.api as sm

import requests
import os

import datetime as dt
import time
# from sqlalchemy import create_engine
# import psycopg2

from pyspark.context import SparkContext
from pyspark.sql import SparkSession

import streamlit as st
from streamlit_jupyter import StreamlitPatcher, tqdm
StreamlitPatcher().jupyter()

pd.set_option('display.max_columns', None)

st.title("Exploratory Data Analysis for GPS Jamming")

In [ ]:
'''df = pd.read_json(os.path.join('aircraft_data_2024-02-01','aircraft_data_2024-02-01.json')
df = pd.json_normalize(df['aircraft'])
df'''

In [ ]:
# for each timeslice of 5 minutes, find the aircraft-ping with nac_p, nac_v above a certain threshold, 
# and gpsOkBefore, rr_lat, rr_lon if these values are not empty
# (each timeslice is 800kb)
#
# after filtering the timeslice, the filtered data should be much smaller say 10 kb
#
# define array of locations (squares) according to lat and lon limits
# 
# loop through all timeslice-aircraft-ping and find location and append
# 
# loop through all locations and run the data through the classification function to determine the color

# url = 'https://samples.adsbexchange.com/readsb-hist/2024/02/01/000000Z.json.gz'
# response = requests.get(url)
# json_data = response.json()
# print(json_data['now'])
# print(json_data['messages'])
# df1 = pd.json_normalize(json_data['aircraft'])


In [ ]:
# spark = SparkSession.builder.getOrCreate()
spark = SparkSession.builder.appName("GPS Jamming").config("spark.memory.offHeap.enabled","true").config("spark.memory.offHeap.size","10g").getOrCreate()
spark.catalog.listTables()

In [ ]:
spark.sql("DROP TABLE flight_data")

In [ ]:
def time_elapsed(start_time):
    end_time = time.time()
    return end_time - start_time

def read_data(start_date_time,end_date_time):
    delta = dt.timedelta(seconds=5)
    start = start_date_time
    while start <= end_date_time:
        # print(start.strftime("%Y/%m/%dT%H%M%SZ"))
        start_time = time.time()
        
        url = f'https://samples.adsbexchange.com/readsb-hist/{start.strftime("%Y/%m/%d")}/{start.strftime("%H%M%S")}Z.json.gz'
        print(url)
    
        response = requests.get(url)
        json_data = response.json()

        print(f"(1): {time_elapsed(start_time)}s")
        start_time = time.time()
        
        utc_time = dt.datetime.utcfromtimestamp(json_data['now'])
        df = pd.json_normalize(json_data['aircraft'])
    
        # Create a new column for the primary key
        df['time'] = utc_time.strftime('%Y%m%dT%H%M%S')
        df = df.fillna("Nan values")
        df = df.applymap(lambda x: ', '.join(x) if isinstance(x, list) else x)
        
        spark_df = spark.createDataFrame(df)

        # Examine the tables in the catalog
        # print(spark.catalog.listTables())

        # Add spark_temp to the catalog
        # spark_df.DataFrameWriter.insertInto("flight_data", overwrite=None)
        spark_df.write.saveAsTable("flight_data", format=None, mode="append")
        start += delta

read_data(dt.datetime(2024,2,1,0,2,0),dt.datetime(2024,2,1,0,10,0))
spark_df = spark.read.table("flight_data")
df1 = spark_df.toPandas()
df1


In [ ]:
print(f"List all type with counts {df1['type'].value_counts()}")
# List all type with counts type
# adsb_icao         9145 - messages from a Mode S or ADS-B transponder, using a 24-bit ICAO address
# other              725 - IGNORE miscellaneous data received via Basestation / SBS format, quality / source is unknown.
# adsb_icao_nt       524 - IGNORE - messages from an ADS-B equipped “non-transponder” emitter e.g. a ground vehicle, using a 24-bit ICAO address
# mode_s             515 - ModeS data from the planes transponder (no position transmitted)
# adsr_icao          280 - rebroadcast of ADS-B messages originally sent via another data link e.g. UAT, using a 24-bit ICAO address
# tisb_other         256 - traffic information about a non-ADS-B target using a non-ICAO address
# tisb_trackfile     214 - traffic information about a non-ADS-B target using a track/file identifier, typically from primary or Mode A/C radar
# mlat               116 - MLAT, position calculated arrival time differences using multiple receivers, outliers and varying accuracy is expected.
# unknown             49
# tisb_icao           31 - traffic information about a non-ADS-B target identified by a 24-bit ICAO address, e.g. a Mode S target tracked by secondary radar
# adsb_other          17 - messages from an ADS-B transponder using a non-ICAO address, e.g. anonymized address
plt.hist(df1['type'], align='left')
plt.title('type')
plt.xticks(rotation='vertical')
plt.show()

In [ ]:

df1_filtered = df1[df1[['nac_p','nac_v']].notnull().all(1)]
nacp_high = df1_filtered[df1_filtered['nac_p']<8]
nacp_high
# rr_lat not null
# rr_lon not null
# lastPosition.lat / lastPosition.lon not null

In [ ]:
print(f"List all ADSB versions with counts {df1['version'].value_counts()}")
# version: ADS-B Version Number 0, 1, 2 (3-7 are reserved) (2.2.3.2.7.5)
# List all ADSB versions with counts version
# 2.0    8881
# 0.0    1119
# 1.0     141
plt.hist(df1['version'])
plt.title('version')
plt.show()

In [ ]:
# nac_p: Navigation Accuracy for Position (2.2.5.1.35)
counts = df1['nac_p'].value_counts()
print(f"List all nac_p values with counts {counts}")
plt.hist(df1['nac_p'])
plt.title('nac_p')
plt.show()

In [ ]:
# nac_v: Navigation Accuracy for Velocity (2.2.5.1.19)
counts = df1['nac_v'].value_counts()
print(f"List all nac_v values with counts {counts}")
plt.hist(df1['nac_v'])
plt.title('nac_v')
plt.show()

In [ ]:
# roll: Roll, degrees, negative is left roll
counts = df1['roll'].value_counts()
print(f"List all roll with counts {counts}")
plt.hist(df1['roll'], align='left')
plt.title('roll')
plt.show()

In [ ]:
# category: emitter category to identify particular aircraft or vehicle classes (values A0 – D7) (2.2.3.2.5.2)
counts = df1['category'].dropna()
# print(f"List all category with counts {counts}")
plt.hist(counts)
plt.title('category')
plt.show()
# A0 : No ADS-B emitter category information. Do not use this emitter category. If no emitter category fits your installation, seek guidance from the FAA as appropriate. A1 : Light (< 15500 lbs) – Any airplane with a maximum takeoff weight less than 15,500 pounds. This includes very light aircraft (light sport aircraft) that do not meet the requirements of 14 CFR § 103.1.
# A2 : Small (15500 to 75000 lbs) – Any airplane with a maximum takeoff weight greater than or equal to15,500 pounds but less than 75,000 pounds.
# A3 : Large (75000 to 300000 lbs) – Any airplane with a maximum takeoff weight greater than or equal to 75,000 pounds but less than 300,000 pounds that does not qualify for the high vortex category.
# A4 :  High vortex large (aircraft such as B-757) – Any airplane with a maximum takeoff weight greater than or equal to 75,000 pounds but less than 300,000 pounds that has been determined to generate a high wake vortex. Currently, the Boeing 757 is the only example.
# A5 : Heavy (> 300000 lbs) – Any airplane with a maximum takeoff weight equal to or above 300,000 pounds.
# A6 : High performance (> 5g acceleration and 400 kts) – Any airplane, regardless of weight, which can maneuver in excess of 5 G’s and maintain true airspeed above 400 knots.
# A7 : Rotorcraft – Any rotorcraft regardless of weight.
# B0 : No ADS-B emitter category information
# B1 : Glider / sailplane – Any glider or sailplane regardless of weight.
# B2 : Lighter-than-air – Any lighter than air (airship or balloon) regardless of weight.
# B3 : Parachutist / skydiver
# B4 : Ultralight / hang-glider / paraglider – A vehicle that meets the requirements of 14 CFR § 103.1. Light sport aircraft should not use the ultralight emitter category unless they meet 14 CFR § 103.1.
# B5 : Reserved
# B6 : Unmanned aerial vehicle – Any unmanned aerial vehicle or unmanned aircraft system regardless of weight.
# B7 : Space / trans-atmospheric vehicle
# C0 : No ADS-B emitter category information
# C1 : Surface vehicle – emergency vehicle
# C2 : Surface vehicle – service vehicle
# C3 : Point obstacle (includes tethered balloons)
# C4 : Cluster obstacle
# C5 : Line obstacle
# C6 : Reserved
# C7 : Reserved

In [ ]:
# Change in NIC/NAC/SIL indicates an anomaly, which may be due to any reason

# nic: Navigation Integrity Category (2.2.3.2.7.2.6)
# Table 1: NIC value and corresponding size of containment radius
# NIC Containment Radius
# 0 Unknown
# 1 Rc < 37.04km (20nm)
# 2 Rc < 14.816km (8nm)
# 3 Rc < 7.408km (4nm)
# 4 Rc < 3.704km (2nm)
# 5 Rc < 1852m (1nm)
# 6 Rc < 1111.2m (0.6nm)
# Rc < 926m (0.5nm)
# Rc < 555.6m (0.3nm)
# 7 Rc < 370.4m (0.2nm)
# 8 Rc < 185.2m (0.1nm)
# 9 Rc < 75m
# 10 Rc < 25m
# 11 Rc < 7.5m

# Assuming you have already calculated the counts
counts = df1['nic'].value_counts(dropna=False)
print(f"List all nic values with counts {counts}")
# Create bins for the histogram
bins = np.arange(len(counts) + 1)

# Plot the histogram with NA on the left
plt.hist(df1['nic'], bins=bins, align='left',rwidth=0.5)
plt.yscale('log')
plt.title('nic')
plt.show()

In [ ]:
counts = df1['sil'].value_counts(dropna=False)

print(f"List all sil values with counts {counts}")
# Create bins for the histogram
bins = np.arange(len(counts) + 1)

# Plot the histogram with NA on the left
plt.hist(df1['sil'], bins=bins, align='left',rwidth=0.5)
plt.yscale('log')
plt.title('sil')
plt.show()

In [ ]:
url2 = 'https://samples.adsbexchange.com/readsb-hist/2024/02/01/000005Z.json.gz'
response2 = requests.get(url2)
json_data2 = response2.json()
df2 = pd.json_normalize(json_data2['aircraft'])
url3 = 'https://samples.adsbexchange.com/readsb-hist/2024/02/01/000010Z.json.gz'
response3 = requests.get(url3)
json_data3 = response2.json()
df3 = pd.json_normalize(json_data3['aircraft'])
url4 = 'https://samples.adsbexchange.com/readsb-hist/2024/02/01/000015Z.json.gz'
response4 = requests.get(url4)
json_data4 = response2.json()
df4 = pd.json_normalize(json_data4['aircraft'])